# Preprocess acoustic data and sync to neural data

Notebook within the chronic ephys processing pipeline
- **1-preprocess_acoustics**
- 2-curate_acoustics
- 3-sort_spikes
- 4-curate_spikes

Use the environment **songproc** to run this notebook

In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import os
import pickle
from scipy.io import wavfile
import traceback

import sys
sys.path.append('/mnt/cube/lo/envs/ceciestunepipe')
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb
from ceciestunepipe.util import stimutil as su
from ceciestunepipe.util import sglxutil as sglu
from ceciestunepipe.util import sglxsync as sy
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import spikeglxrecordingextractor as sglex
from ceciestunepipe.util import oeutil as oeu
from ceciestunepipe.mods import preproc_sglx, preproc_oe

import logging
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.WARNING) # set to logging.INFO if you'd like to see the full readout

/mnt/cube/lo/envs/songproc/lib/python3.8/site-packages/spikeextractors/__init__.py:21: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if StrictVersion(h5py.__version__) > '2.10.0':
2024-11-14 10:00:00,473 root         INFO     Running on pakhi.ucsd.edu


h5py version > 2.10.0. Some extractors might not work properly. It is recommended to downgrade to version 2.10.0: 
>>> pip install h5py==2.10.0


## Set parameters

In [2]:
## default bout detection parameters that work well for zebra finches
hparams = {
    # spectrogram
    'num_freq':1024, # how many channels to use in a spectrogram
    'preemphasis':0.97,
    'frame_shift_ms':5, # step size for fft
    'frame_length_ms':10, # frame length for fft FRAME SAMPLES < NUM_FREQ!!!
    'min_level_db':-55, # minimum threshold db for computing spectrogram
    'ref_level_db':110, # reference db for computing spectrogram
    'sample_rate':None, # sample rate of your data
    
    # mel filter
    'mel_filter':False, # should a mel filter be used?
    'num_mels':1024, # how many channels to use in the mel-spectrogram
    'fmin':300, # low frequency cutoff for mel filter
    'fmax':12000, # high frequency cutoff for mel filter
    
    # spectrogram inversion
    'max_iters':200,
    'griffin_lim_iters':20,
    'power':1.5,
    
    # bout searching
    'bout_auto_file':'bout_auto.pickle', # extension for saving the auto found files
    'bout_sync_file':'bout_sync.pickle', # extension for saving the synchronized auto bouts
    'stim_sync_file':'stim_sync.pickle', # extension for saving the synchronized stim if stim session
    'bout_curated_file':'bout_curated.pickle', # extension for manually curated files
    
    # if using deep_bout_search = False, the following parameters will apply for automatic bout detection:
    'read_wav_fun':bs.read_npy_chan, # function for loading the wav_like_stream (returns fs, ndarray)
    'file_order_fun':bs.sess_file_id, # function for extracting the file ID within the session
    'min_segment':20, # minimum length of supra_threshold to consider a 'syllable' (ms)
    'min_silence':3000, # minmum distance between groups of syllables to consider separate bouts (ms)
    'min_bout':500, # min bout duration (ms)
    'peak_thresh_rms':0.55, # threshold (rms) for peak acceptance,
    'thresh_rms':0.25, # threshold for detection of syllables
    'mean_syl_rms_thresh':0.3, # threshold for acceptance of mean rms across the syllable (relative to rms of the file)
    'max_bout':180000, # exclude bouts too long (ms)
    'l_p_r_thresh':100, # threshold for n of len_ms/peaks (typycally about 2-3 syllable spans)
    'waveform_edges':1000, # get number of ms before and after the edges of the bout for the waveform sample
}

## other processing parameters
n_jobs = 1 # n_jobs for deriving bout info (errors when increased)
mic_file_ext = 'npy' # npy method more efficient than wav
force_preprocess = False # skip preprocessing for previously failed epochs
deep_bout_search = True # detect bouts using deep search -- see ceciestunepipe.mods.bout_detection_mf

In [3]:
# dictionaries for birds / recordings
bird_rec_dict = {
###### Zeke/Pablo had 3 successful finch recordings in the past: z_w12m7_20, z_r12r13_21, z_y19o20_21
#     'z_w12m7_20':[
#         {'sess_par_list':['2020-11-04'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_0'], # list of mics of interest, by signal name in rig.json
#          'adc_list':[], # list of adc channels of interest
#          'stim_list':[], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to syncrhonize everything to (sglx only, oe already synced)
#          'trial_tag_chan':[], # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':[], # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort index
#          'ephys_software':'sglx' # sglx or oe
#         }],
#    'z_r12r13_21':[
#        {'sess_par_list':['2021-06-27','2021-06-28'], # sessions with this configuration
#         'stim_sess_list':[], # sessions where stimuli were presented
#         'mic_list':['microphone_0'], # list of mics of interest, by signal name in rig.json
#         'adc_list':[], # list of adc channels of interest
#         'stim_list':[], # list of adc chans with the stimulus
#         'nidq_ttl_list': [], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#         'ref_stream':'ap_0', # what to syncrhonize everything to (sglx only, oe already synced)
#         'trial_tag_chan':[], # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#         'on_signal':[], # sglx, whether signal on is hi or lo
#         'sort':'sort_0', # sort index
#         'ephys_software':'sglx' # sglx or oe
#        }], 
#     'z_y19o20_21':[   # two implants, 10/26-27 and 12/21-23, with same rig.json info (just different probe serial #)
#         {'sess_par_list':['2021-10-27'],# ['2021-10-26','2021-10-27','2021-12-21','2021-12-22','2021-12-23'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'probes':['probe_0'],
#          'mic_list':['microphone_1'], # list of mics of interest, by signal name in rig.json
#          'adc_list':[], # list of adc channels of interest
#          'stim_list':[], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to syncrhonize everything to (sglx only, oe already synced)
#          'trial_tag_chan':[], # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':[], # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort index
#          'ephys_software':'sglx' # sglx or oe
#         }]
###### Pablo and I also collected another finch's data: z_c5o30_23
    ## Session 2023-06-12 was directly after the surgery, no song collected
    ##   -- Epoch 1752: neuropixel came unplugged, data length mismatch
    ## Session 2023-06-15 is the best session (according to Pablo)
#     'z_c5o30_23':[
#         {'sess_par_list':['2023-06-15'], # sessions with this configuration '2023-06-12','2023-06-13','2023-06-14','2023-06-15','2023-06-15'
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_0'], # list of mics of interest, by signal name in rig.json
#          'adc_list':[], # list of adc channels of interest
#          'stim_list':[], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
#          'trial_tag_chan':0, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':1, # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort index
#          'ephys_software':'sglx' # sglx or oe
#         }]
###### My data:
#     'z_c4y1_23':[
#         {'sess_par_list':['2024-02-08','2024-02-09','2024-02-10','2024-02-11','2024-02-12'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
#          'adc_list':[], # list of adc channels of interest
#          'stim_list':['wav_stim','wav_syn'], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
#          'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':1, # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort info
#          'ephys_software':'sglx' # sglx or oe
#         }],
#     'z_g9y18_23':[
#         {'sess_par_list':['2024-04-17','2024-04-18','2024-04-19'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
#          'adc_list':[], # list of adc channels of interest
#          'stim_list':['wav_stim','wav_syn'], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
#          'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':1, # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort info
#          'ephys_software':'sglx' # sglx or oe
#         }],
#     'z_p5y10_23':[
#         {'sess_par_list':['2024-05-16','2024-05-17'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
#          'adc_list':[], # list of adc channels of interest
#          'stim_list':['wav_stim','wav_syn'], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
#          'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':1, # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort info
#          'ephys_software':'sglx' # sglx or oe
#         }]
#     'z_r5r13_24':[
#         {'sess_par_list':['2024-08-05','2024-08-05_reduced_chans','2024-08-06','2024-08-07','2024-08-08'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
#          'adc_list':[], # list of adc channels of interest
#          'stim_list':['wav_stim','wav_syn'], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
#          'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':1, # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort info
#          'ephys_software':'sglx' # sglx or oe
#         }]
#     'z_g10r16_24':[
#         {'sess_par_list':['2024-09-24','2024-09-25','2024-09-26','2024-09-26_overnight','2024-09-27'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
#          'adc_list':[], # list of adc channels of interest
#          'stim_list':['wav_stim','wav_syn'], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
#          'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':1, # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort info
#          'ephys_software':'sglx' # sglx or oe
#         }]
#     'z_p6p10_24':[
#         {'sess_par_list':['2024-10-01','2024-10-02','2024-10-03'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
#          'adc_list':['opto_stim'], # list of adc channels of interest
#          'stim_list':['wav_stim','wav_syn'], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
#          'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':1, # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort info
#          'ephys_software':'sglx' # sglx or oe
#         }]
#     'z_r7c3_23':[
#         {'sess_par_list':['2024-06-03','2024-06-04'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
#          'adc_list':[], # list of adc channels of interest
#          'stim_list':['wav_stim','wav_syn'], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
#          'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':1, # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort info
#          'ephys_software':'sglx' # sglx or oe
#         }]
#     'z_r1p1_24':[
#         {'sess_par_list':['2024-10-18','2024-10-19'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
#          'adc_list':[], # list of adc channels of interest
#          'stim_list':[], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
#          'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':1, # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort info
#          'ephys_software':'sglx' # sglx or oe
#         }]
#     'z_c7r3_24':[
#         {'sess_par_list':['2024-10-26','2024-10-27'], # sessions with this configuration
#          'stim_sess_list':[], # sessions where stimuli were presented
#          'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
#          'adc_list':['opto_stim'], # list of adc channels of interest
#          'stim_list':[], # list of adc chans with the stimulus
#          'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
#          'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
#          'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
#          'on_signal':1, # sglx, whether signal on is hi or lo
#          'sort':'sort_0', # sort info
#          'ephys_software':'sglx' # sglx or oe
#         }]
    'z_b1w4_24':[
        {'sess_par_list':['2024-11-11','2024-11-12','2024-11-13'], # sessions with this configuration
         'stim_sess_list':[], # sessions where stimuli were presented
         'mic_list':['microphone_M','microphone_F'], # list of mics of interest, by signal name in rig.json
         'adc_list':[], # list of adc channels of interest
         'stim_list':['wav_stim','wav_syn'], # list of adc chans with the stimulus
         'nidq_ttl_list':[], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
         'ref_stream':'ap_0', # what to synchronize everything to (sglx only, oe already synced)
         'trial_tag_chan':2, # sglx, what was the tag channel in the stimulus wave (this should come from meta et. al)
         'on_signal':1, # sglx, whether signal on is hi or lo
         'sort':'sort_0', # sort info
         'ephys_software':'sglx' # sglx or oe
        }]
}

## Preprocess and synchronize recordings

In [4]:
%%time

# loop through all birds / recordings
for this_bird in bird_rec_dict.keys():
    # get session parameters
    sess_par_all = bird_rec_dict[this_bird]
    # get all sessions
    all_bird_sess = et.list_sessions(this_bird)
    
    # loop through sessions
    for this_sess in all_bird_sess:
        
        # loop though possible session parameter sets
        sess_par_match = []
        for this_sess_par in sess_par_all:
            # save if there is a match
            if this_sess in this_sess_par['sess_par_list']:
                sess_par_match.append(this_sess_par)      
        
        # ensure only one match
        if len(sess_par_match) == 1:
            sess_par = sess_par_match[0]
            # add bird and sess variables
            sess_par['bird'] = this_bird        
            sess_par['sess'] = this_sess
            # stim session
            if this_sess in this_sess_par['stim_sess_list']:
                sess_par['stim_sess'] = this_sess
            else:
                sess_par['stim_sess'] = []
            
            # get experiment structure
            exp_struct = et.get_exp_struct(sess_par['bird'],sess_par['sess'],sort=sess_par['sort'],ephys_software=sess_par['ephys_software'])
            
            # get preprocessing log
            log_dir = os.path.join('/mnt/cube/chronic_ephys/log', this_bird, this_sess)
            log_file = os.path.join(log_dir,'preprocessing.log')
            try:
                with open(log_file, 'r') as f:
                    log_message=f.readline() # read the first line of the log file
                if log_message[:-1] == sess_par['bird']+' '+sess_par['sess']+' preprocessing complete without error':
                    print(sess_par['bird'],sess_par['sess'],'already exists -- skipping preprocessing')
                    run_preprocess = False
                elif log_message[:-1] == sess_par['bird']+' '+sess_par['sess']+' preprocessing failed':
                    if force_preprocess:
                        print(sess_par['bird'],sess_par['sess'],'failed previously -- running preprocessing')
                        run_preprocess = True
                    else:
                        print(sess_par['bird'],sess_par['sess'],'failed previously -- skipping preprocessing')
                        run_preprocess = False
                else: # uninterpretable log file
                    print(sess_par['bird'],sess_par['sess'],'couldn\'t interpret log file -- running preprocessing')
                    run_preprocess = True
            except: # no existing log file
                print(sess_par['bird'],sess_par['sess'],'no log file found -- running preprocessing')
                run_preprocess = True
            
            # run preprocessing 
            if run_preprocess:
                
                ##### preprocess acoustics #####
                print(this_bird,this_sess,sess_par['ephys_software'],'preprocessing session..')
                if sess_par['ephys_software'] == 'sglx':
                    preproc_sglx.preprocess_session(sess_par,force_redo=True)
                elif sess_par['ephys_software'] == 'oe':
                    preproc_oe.preprocess_session(sess_par,force_redo=True)
                
                ###### derive bout information #####
                print(this_bird,this_sess,'deriving bout information..')
                sess_bout_pd = sb.get_all_day_bouts(sess_par,hparams,n_jobs=n_jobs,ephys_software=sess_par['ephys_software'],
                                                    file_ext=mic_file_ext, deep_search=deep_bout_search)
                
                # get epochs
                sess_epochs = et.list_ephys_epochs(sess_par)
                
                # store bout info for each epoch
                bout_syn_pd_all = []
                # stim sess collect all stim as well
                if len(sess_par['stim_sess']) > 0:
                    trial_syn_pd_all = []
                
                # loop through epochs
                epoch_fail = np.zeros_like(sess_epochs, dtype=bool) # track failed preprocessing per epoch
                error_tracker = {}
                for epoch_n,this_epoch in enumerate(sess_epochs):
                    try:
                        print(this_bird,this_sess,this_epoch,'syncing..')
                        # add to session parameter dictionary
                        sess_par['epoch'] = this_epoch
                        # get epoch info
                        epoch_struct = et.sgl_struct(sess_par,this_epoch,ephys_software=sess_par['ephys_software'])

                        ##### synchronization - sglx #####
                        if sess_par['ephys_software'] == 'sglx':
                            # get epoch files
                            sgl_folders, sgl_files = sglu.sgl_file_struct(epoch_struct['folders']['sglx'])
                            run_meta_files = {k:v[0] for k,v in sgl_files.items()}
                            run_recordings = {k:sglex.SpikeGLXRecordingExtractor(sglu.get_data_meta_path(v)[0]) for k,v in run_meta_files.items()}

                            # get streams, from raw recording extractors and preprocessed data
                            all_streams = list(run_recordings.keys()) + ['wav'] ### might want to just remove this
                            # get sync pattern
                            all_syn_dict = {k:sy.get_syn_pattern(run_recordings,epoch_struct,k,force=False) for k in all_streams}
                            # run sync
                            sy.sync_all(all_syn_dict,sess_par['ref_stream'],force=False)

                            # load bouts
                            hparams, bout_pd = sb.load_bouts(sess_par['bird'],sess_par['sess'],'', derived_folder='bouts_sglx',bout_file_key='bout_auto_file')
                            # keep only epoch bouts
                            logger.info('bouts from this epoch {}'.format(sess_par['epoch']))
                            drop_condition = ~bout_pd['file'].str.contains(sess_par['epoch'])
                            bout_pd.drop(bout_pd[drop_condition].index, inplace=True)
                            bout_pd.reset_index(drop=True, inplace=True)
                            # sync bouts to spike time base
                            bout_dict, bout_syn_pd = sy.bout_dict_from_pd(bout_pd,all_syn_dict,s_f_key='wav')
                            # store epoch synced bout info
                            bout_syn_pd['bird'] = this_bird
                            bout_syn_pd['sess'] = this_sess
                            bout_syn_pd['epoch'] = this_epoch
                            bout_syn_pd_all.append(bout_syn_pd)
                            # save synced bouts
                            bout_dict_path = os.path.join(epoch_struct['folders']['derived'],'bout_dict_ap0.pkl')
                            with open(bout_dict_path, 'wb') as handle:
                                pickle.dump(bout_dict, handle)
                            bout_pd_path = os.path.join(epoch_struct['folders']['derived'],'bout_pd_ap0.pkl')
                            bout_pd.to_pickle(bout_pd_path)
                            logger.info('saved syncronized bout dict and pandas dataframe to {}, {}'.format(bout_dict_path, bout_pd_path))

                            if len(sess_par['stim_sess']) > 0:
                                # syn_ttl comes from the digital pin, syn_sine_ttl from the sine
                                event_name = 'wav_stim'
                                ttl_ev_name = event_name + '_sync_sine_ttl' 
                                # get the events npy file
                                npy_stim_path = os.path.join(epoch_struct['folders']['derived'],ttl_ev_name + '_evt.npy')
                                stream_stim_path = os.path.join(epoch_struct['folders']['derived'],event_name + '.npy')
                                trial_ttl = np.load(npy_stim_path)
                                # epoch may not have trials - if so ttl file will be empty
                                if len(trial_ttl) > 0:
                                    trial_stream = np.load(stream_stim_path,mmap_mode='r')
                                    # get sampling frequency
                                    stim_s_f = int(all_syn_dict['nidq']['s_f'])
                                    # load the stimulus name - frequency tag dictionary
                                    stim_tags_dict = preproc_sglx.load_stim_tags_dict(sess_par['stim_sess'],sess_par['bird'])
                                    # get trial tagged dataframe
                                    trial_tagged_pd = su.get_trials_pd(trial_ttl, trial_stream, stim_s_f,on_signal=sess_par['on_signal'],
                                                                       tag_chan=sess_par['trial_tag_chan'],stim_tags_dict=stim_tags_dict,
                                                                       trial_is_onof=True)
                                    # sync stim
                                    trial_dict, trial_syn_pd = sy.trial_syn_from_pd(trial_tagged_pd,all_syn_dict,s_f_key='nidq')
                                    # store epoch synced stim info
                                    trial_syn_pd['bird'] = this_bird
                                    trial_syn_pd['sess'] = this_sess
                                    trial_syn_pd['epoch'] = this_epoch
                                    trial_syn_pd_all.append(trial_syn_pd)
                                    # save synced stim
                                    stim_dict_path = os.path.join(epoch_struct['folders']['derived'],'stim_dict_ap0.pkl')
                                    stim_pd_path = os.path.join(epoch_struct['folders']['derived'],'stim_pd_ap0.pkl')
                                    with open(stim_dict_path,'wb') as handle:
                                        pickle.dump(trial_dict,handle)
                                    trial_syn_pd.to_pickle(stim_pd_path)
                                    logger.info('saved syncronized stim dict and pandas dataframe to {}, {}'.format(stim_dict_path, stim_pd_path))

                        ###### synchronization - oe #####
                        elif sess_par['ephys_software'] == 'oe':
                            # get epoch files
                            run_recordings = {'oeb':preproc_oe.get_oe_cont_recording(exp_struct,this_epoch)}

                            # make an all_syn_dict
                            mic_file_name = os.path.join(exp_struct['folders']['derived'],this_epoch,'wav_mic-npy_meta.pickle')
                            with open(mic_file_name, 'rb') as handle:
                                wav_mic_meta = pickle.load(handle)
                            all_syn_dict = {'wav': {'s_f':wav_mic_meta['s_f']}, 
                                           'ap_0': {'s_f':run_recordings['oeb'].get_sampling_frequency()},
                                           'nidq': {'s_f':run_recordings['oeb'].get_sampling_frequency()}}
                            # make bouts pandas file for this session - match sglx format, streams already synced
                            bout_oe_struct = et.get_exp_struct(sess_par['bird'],sess_par['sess'],sort=sess_par['sort'],ephys_software='bouts_oe')
                            bout_pd_path = os.path.join(bout_oe_struct['folders']['derived'], 'bout_auto.pickle')
                            bout_syn_pd = pd.read_pickle(bout_pd_path)
                            bout_dict = preproc_oe.bout_dict_from_pd(bout_syn_pd,all_syn_dict)
                            # store epoch synced bout info
                            bout_syn_pd['bird'] = this_bird
                            bout_syn_pd['sess'] = this_sess
                            bout_syn_pd['epoch'] = this_epoch
                            bout_syn_pd_all.append(bout_syn_pd)
                            # save synced bouts
                            bout_dict_path = os.path.join(epoch_struct['folders']['derived'],'bout_dict_oe.pkl')
                            bout_pd_path = os.path.join(epoch_struct['folders']['derived'],'bout_pd_oe.pkl')
                            with open(bout_dict_path,'wb') as handle:
                                pickle.dump(bout_dict,handle)
                            bout_syn_pd.to_pickle(bout_pd_path)

                            if len(sess_par['stim_sess']) > 0:
                                # this epoch name - get recording events path
                                raw_folder = exp_struct['folders']['oe']
                                epoch_path = os.path.join(raw_folder,this_epoch)
                                node_path = preproc_oe.get_default_node(exp_struct,this_epoch)
                                rec_path = preproc_oe.get_default_recording(node_path)
                                events_path = os.path.join(rec_path,'events/Network_Events-102.0/TEXT_group_1/')
                                # load stim lables / onsets
                                stim_labels = np.load(os.path.join(events_path,'text.npy'))
                                stim_onsets = np.load(os.path.join(events_path,'timestamps.npy'))

                                # get stim onsets and offsets
                                stim_on_all = []; stim_off_all = []; 
                                stim_proc_path_all = []; stim_exp_path_all = [];
                                stim_map_dir_all = []; stim_id_all = [];
                                # loop through stim
                                for stim_i in range(len(stim_labels)):
                                    this_stim_label = stim_labels[stim_i].astype('str')
                                    this_stim_onset = stim_onsets[stim_i]
                                    if this_stim_label[:4] == 'stim':
                                        stim_exp_file = this_stim_label[5:]
                                        # get stim preprocessing directory
                                        stim_file_split = stim_exp_file.split('/')
                                        stim_map_i = np.where([stim_file_split[i] in list(stim_map_dict.keys()) for i in range(len(stim_file_split))])[0][0]
                                        stim_map_dir = stim_map_dict[stim_file_split[stim_map_i]]
                                        # get remaining stim file path - identical for experiment and preprocessing
                                        remaining_stim_file = '/'.join(stim_file_split[stim_map_i+1:])
                                        # processing file location
                                        stim_file = os.path.join(stim_map_dir,remaining_stim_file)
                                        # load stim and get length
                                        sf,this_wav = wavfile.read(stim_file,mmap=True)
                                        stim_len = this_wav.shape[0]/sf
                                        # get length of stim in samples - round up
                                        stim_samp_len = int(np.ceil(stim_len * bout_dict['s_f']))
                                        # get stim on / off
                                        stim_on_all.append(this_stim_onset)
                                        stim_off_all.append(this_stim_onset+stim_samp_len)  
                                        stim_proc_path_all.append(stim_file)
                                        stim_exp_path_all.append(stim_exp_file)
                                        stim_map_dir_all.append(stim_map_dir)
                                        stim_id_all.append(remaining_stim_file)

                                # make into a pd - oe already synced
                                stim_on_all_np = np.array(stim_on_all).astype('int')
                                stim_off_all_np = np.array(stim_off_all).astype('int')
                                stim_on_all_np_ms = 1000*(stim_on_all_np/bout_dict['s_f'])
                                stim_off_all_np_ms = 1000*(stim_off_all_np/bout_dict['s_f'])
                                trial_syn_pd = pd.DataFrame(np.vstack([stim_on_all_np,
                                                                    stim_off_all_np,
                                                                    stim_on_all_np_ms,
                                                                    stim_off_all_np_ms,
                                                                    stim_off_all_np_ms-stim_on_all_np_ms,
                                                                    stim_proc_path_all,
                                                                    stim_exp_path_all,
                                                                    stim_map_dir_all,
                                                                    stim_id_all]).T,
                                columns=['start_sample','end_sample','start_ms','end_ms','len_ms',
                                         'proc_file','exp_file','map_dir','stim_id'])
                                trial_syn_pd['start_sample'] = trial_syn_pd['start_sample'].astype('int')
                                trial_syn_pd['end_sample'] = trial_syn_pd['end_sample'].astype('int')
                                trial_syn_pd['start_ms'] = trial_syn_pd['start_ms'].astype('float')
                                trial_syn_pd['len_ms'] = trial_syn_pd['len_ms'].astype('float')
                                # store epoch synced stim info
                                trial_syn_pd['bird'] = this_bird
                                trial_syn_pd['sess'] = this_sess
                                trial_syn_pd['epoch'] = this_epoch
                                trial_syn_pd_all.append(trial_syn_pd)
                                trial_dict = {
                                    's_f': all_syn_dict['wav']['s_f'],
                                    'ap_0':all_syn_dict['ap_0']['s_f'],
                                    'nidq':all_syn_dict['nidq']['s_f'],
                                    'start_ms':trial_syn_pd['start_ms'],
                                    'len_ms':trial_syn_pd['len_ms'],
                                    'start_sample':trial_syn_pd['start_sample'],
                                    'end_sample':trial_syn_pd['end_sample'],
                                    'proc_file':trial_syn_pd['proc_file'],
                                    'exp_file':trial_syn_pd['exp_file'],
                                    'map_dir':trial_syn_pd['map_dir'],
                                    'stim_id':trial_syn_pd['stim_id']}
                                # save synced stim
                                stim_dict_path = os.path.join(epoch_struct['folders']['derived'],'stim_dict_ap0.pkl')
                                stim_pd_path = os.path.join(epoch_struct['folders']['derived'],'stim_pd_ap0.pkl')
                                with open(stim_dict_path,'wb') as handle:
                                    pickle.dump(trial_dict,handle)
                                trial_syn_pd.to_pickle(stim_pd_path)
                                logger.info('saved syncronized stim dict and pandas dataframe to {}, {}'.format(stim_dict_path, stim_pd_path))
                                    
                    # record failed epoch 
                    except Exception as e:
                        epoch_fail[epoch_n] = True
                        error_tracker[this_epoch] = f"Error: {str(e)}"
                        
                if not np.any(epoch_fail): # if preprocessing complete without error..
                    print(sess_par['bird']+' '+sess_par['sess']+' preprocessing complete without error')
                    
                    # concatenate list of synced bout data frames from each epoch and save
                    bout_syn_pd_all_cat = pd.concat(bout_syn_pd_all)
                    sb.save_auto_bouts(bout_syn_pd_all_cat,sess_par,hparams,software=sess_par['ephys_software'],bout_file_key='bout_sync_file')

                    # stim sess save the all sync epoch stim data frame as well
                    if len(sess_par['stim_sess']) > 0:
                        trial_syn_pd_all_cat = pd.concat(trial_syn_pd_all)
                        sb.save_auto_bouts(trial_syn_pd_all_cat,sess_par,hparams,software=sess_par['ephys_software'],bout_file_key='stim_sync_file')
                    
                    # log preprocessing complete without error
                    if not os.path.exists(log_dir): os.makedirs(log_dir)
                    with open(log_file, 'w') as f:
                        f.write(sess_par['bird']+' '+sess_par['sess']+' preprocessing complete without error\nEpochs '+', '.join(sess_epochs)+' processed\n')
                else:
                    # log failed epochs and exit without saving
                    failed_epochs = np.array(sess_epochs)[epoch_fail]
                    if not os.path.exists(log_dir): os.makedirs(log_dir)
                    with open(log_file, 'w') as f:
                        f.write(sess_par['bird']+' '+sess_par['sess']+' preprocessing failed\nProblematic epoch(s): '+', '.join(failed_epochs)+'\n')
                        for err_epoch, err_msg in error_tracker.items():
                            f.write(err_epoch+': '+err_msg+'\n')
                    #### common reason for failure: data length mismatch (ex. mic or neuropixel came unplugged)

z_b1w4_24 2024-11-11 already exists -- skipping preprocessing
z_b1w4_24 2024-11-12 already exists -- skipping preprocessing
z_b1w4_24 2024-11-13 no log file found -- running preprocessing
z_b1w4_24 2024-11-13 sglx preprocessing session..
preprocessing..
z_b1w4_24 2024-11-13 deriving bout information..


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

z_b1w4_24 2024-11-13 0700_g0 syncing..
z_b1w4_24 2024-11-13 1000_g0 syncing..
z_b1w4_24 2024-11-13 1301_g0 syncing..
z_b1w4_24 2024-11-13 preprocessing complete without error
CPU times: user 1h 22min 52s, sys: 41min 2s, total: 2h 3min 54s
Wall time: 3h 1min 36s
